In [1]:
import numpy as np
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt

In [4]:
torch.randint(0,10,size=(64,)).shape

torch.Size([64])

In [4]:
from DiffusionModel import DiffusionUNet
resolution = 32
net = DiffusionUNet(in_channels=3, resolution=32, attn_resolutions=[16], ch_mult=(2,2,2), channels=64, device="cpu")

In [5]:
B = 32
x = torch.randn((B,3,resolution,resolution))
t = torch.ones((B), dtype=torch.int32)

In [13]:
print(x.flatten(start_dim=2).mT.shape)

torch.Size([32, 1024, 3])


In [6]:
net.sinusoidal_embedding(x, t).shape

torch.Size([32, 256])

In [7]:
net(x, t).shape

torch.Size([32, 3, 32, 32])

In [ ]:
from DiffusionModel import DownBlock, UpBlock

B = 32
x = torch.randn((B,64,64,64))
t = torch.randn((B,256))

d1 = DownBlock(64, 128)
d2 = DownBlock(128, 256)
d3 = DownBlock(256, 512)

x_1, skips_1 = d1(x, t)
x_2, skips_2 = d2(x_1, t)
x_3, skips_3 = d3(x_2, t)

print(x_3.shape)


In [ ]:
print(skips_3[0].shape)

In [ ]:
x_3.shape

In [ ]:

u3 = UpBlock(512*2, 256)
u2 = UpBlock(256*2, 128)
u1 = UpBlock(128*2, 64)


y_3 = u3( x_3, t, skips_3 )
y_2 = u2( y_3, t, skips_2 )
y_1 = u1( y_2, t, skips_1 )
print(y_1.shape)

In [ ]:
my_conv = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
my_conv(x).shape

In [ ]:
from FastGan import SLEBlock, UpSampleBlock


upblock = UpSampleBlock(in_channels=512, out_channels=512)
sleblock = SLEBlock(size=8, low_channels=512, high_channels=64)

In [ ]:
x =  torch.randn((1,512,4,4))
up_x = upblock(x)
print(up_x.shape)
sle_x = sleblock(up_x, torch.randn(1,64,128,128))
print(sle_x.shape)


In [ ]:
class EqualizedConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, bias=True):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias)
        # initialize weights to N(0,1)
        self.conv.weight.data.normal_(0,1)
        self.conv.bias.data.fill_(0)
    
    def forward(self,x):
        x = self.conv(x)*self.get_scale()
        return x
    
    def get_scale(self):
        size_w = self.conv.weight.size()
        fan_in = size_w[1:].numel()
        return np.sqrt(2/fan_in)

In [ ]:
out_ch = 512
in_ch = 512
k = (4,4)
shape = (1,1)
x = torch.randn((1,out_ch,*shape))
print(x.shape)

In [ ]:
conv = EqualizedConv2d(out_ch, in_ch, k, padding=3)
conv2 = EqualizedConv2d(in_ch, in_ch, kernel_size=3, padding=1)

In [ ]:
conv.conv.weight.std()

In [ ]:
out_x = conv(x)
print(out_x.shape)

In [ ]:
conv2(out_x).shape

In [ ]:
from PGGAN import PixelWiseNorm

class GblockPGGAN(nn.Module):
    def __init__(self, in_channels, out_channels, first_block=False):
        super().__init__()
        if first_block:
            self.block = nn.Sequential(
                    EqualizedConv2d(in_channels, out_channels, kernel_size=4, padding=3),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    )
        else:
            self.block = nn.Sequential(
                    nn.Upsample(scale_factor=2, mode='nearest'),
                    EqualizedConv2d(in_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    )
    def forward(self, x):
        return self.block(x)    

In [ ]:
block1 = GblockPGGAN(in_channels=512, out_channels=512, first_block=True)
block2 = GblockPGGAN(in_channels=512, out_channels=512, first_block=False)
block3 = GblockPGGAN(in_channels=512, out_channels=256, first_block=False)

In [ ]:
x = torch.randn((1,512,1,1))
print(block1(x).shape)
print(block2(block1(x)).shape)
print(block3(block2(block1(x))).shape)

In [ ]:
to_rgb = EqualizedConv2d(256, 3, 1, padding=0)

In [ ]:
to_rgb(block3(block2(block1(x)))).shape

In [ ]:
class GeneratorPGGAN(nn.Module):
    def __init__(self, latent_dim=512, out_scale=256):
        super().__init__()

        self.depth = 1
        self.alpha = 1
        self.step_alpha = 0

        self.upscale_blocks = nn.ModuleList()
        self.to_rgb = nn.ModuleList()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        # add first block to get 4x4 image
        self.upscale_blocks.append(GblockPGGAN(in_channels=latent_dim, out_channels=latent_dim, first_block=True))
        self.to_rgb.append( EqualizedConv2d(in_channels=latent_dim, out_channels=3, kernel_size=1, padding=0), )

        start_scale = np.log2(4)
        end_scale = np.log2(out_scale)

        start_scale = int(np.log2(4))
        end_scale   = int(np.log2(out_scale))

        in_ch, out_ch = latent_dim, latent_dim
        for i in range(start_scale+1, end_scale+1):
            in_ch = out_ch
            if i >= 6:
                out_ch = in_ch//2
            else:
                out_ch = in_ch
            self.upscale_blocks.append(GblockPGGAN(in_channels=in_ch, out_channels=out_ch, first_block=False))
            self.to_rgb.append( EqualizedConv2d(in_channels=out_ch, out_channels=3, kernel_size=1, padding=0), )
    
    def forward(self, x):
        # go until second last layer
        for block in self.upscale_blocks[:self.depth-1]:
            x = block(x)
        # compute last layer
        x_prev = x.clone().detach()
        print(f"before {x.shape}")
        x_rgb = self.upscale_blocks[self.depth-1](x)
        print(f"after {x.shape}")
        print(f"are equal? {torch.equal(x,x_prev)}")
        x_rgb = self.to_rgb[self.depth-1](x_rgb)
        #print(f"last layer    x_rgb {x_rgb.shape}   {x.shape}")
        # smooth change
        if self.alpha < 1 and self.depth > 1: # for the first layer cannot be applied
            x_old_rgb = self.upsample(x)
            x_old_rgb = self.to_rgb[self.depth-2](x_old_rgb)
            #print(f"x_old {x_old_rgb.shape}")
            
            x_rgb = self.alpha * x_rgb + (1-self.alpha) * x_old_rgb
        return x_rgb
    
    def increase_net(self, n_iterations):
        self.step_alpha = 1/n_iterations
        self.alpha = 1/n_iterations
        self.depth += 1
    
    def step(self):
        # increase alpha
        self.alpha += self.step_alpha

In [ ]:
g = GeneratorPGGAN()

In [ ]:
layers = list(dict(g.upscale_blocks[0].block.named_children()).values())
print(layers[0].conv.bias)

In [ ]:
x = torch.randn((1,512,1,1))

In [ ]:
g.depth = 1
g.alpha = 0.5
g.step_alpha = 0.1
for i in range(7):
    out_x = g(x)
    print(f"out tensor {out_x.shape}  alpha {g.alpha}")
    g.depth += 1

In [ ]:
from_rgb = nn.Sequential( EqualizedConv2d(in_channels=3, out_channels=512, kernel_size=1), nn.LeakyReLU(0.2))

In [ ]:
x = torch.randn((1,3,256,256))

In [ ]:
from_rgb(x).shape

In [ ]:
start_scale = int(np.log2(4))
end_scale = int(np.log2(256))

for i in reversed(range(start_scale, end_scale+1)):
    print(i, 2**i, 512)

In [ ]:
start_scale = int(np.log2(4))
end_scale   = int(np.log2(256))
latent_dim = 512
out_ch, in_ch = latent_dim, latent_dim
for i in range(start_scale+1, end_scale+1):
    out_ch = in_ch
    if i >= 6:
        in_ch = out_ch//2
    else:
        in_ch = out_ch
    print(f"in {in_ch} out {out_ch}")

In [ ]:
from PGGAN import MinibatchStd

class DblockPGGAN(nn.Module):
    def __init__(self, in_channels, out_channels, last_block=False):
        super().__init__()
        if last_block:
            self.block = nn.Sequential(
                    MinibatchStd(),
                    EqualizedConv2d(in_channels+1, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=4, padding=0),
                    nn.LeakyReLU(0.2, True),
                    nn.Sequential(nn.Flatten(), nn.Linear(out_channels, 1))
                    )
        else:
            self.block = nn.Sequential(
                    EqualizedConv2d(in_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.AvgPool2d(kernel_size=2, stride=2) # down sampling
                    )
            
    def forward(self, x):
        return self.block(x) 

In [ ]:
class DiscriminatorPGGAN(nn.Module):
    def __init__(self, latent_dim=512, out_scale=256):
        super().__init__()

        self.depth = 1
        self.alpha = 1
        self.step_alpha = 0

        self.downscale_blocks = nn.ModuleList()
        self.from_rgb = nn.ModuleList()
        self.downscale = nn.AvgPool2d(kernel_size=2, stride=2)
        self.from_rgb.append( nn.Sequential( EqualizedConv2d(in_channels=3, out_channels=latent_dim, kernel_size=1), nn.LeakyReLU(0.2) ) )
        # 4x4 -> 1x1
        self.downscale_blocks.append(DblockPGGAN(in_channels=latent_dim, out_channels=1, last_block=True))

        start_scale = int(np.log2(4))
        end_scale   = int(np.log2(out_scale))

        out_ch, in_ch = latent_dim, latent_dim
        for i in range(start_scale+1, end_scale+1):
            out_ch = in_ch
            if i >= 6:
                in_ch = out_ch//2
            else:
                in_ch = out_ch
            self.from_rgb.append( nn.Sequential( EqualizedConv2d(in_channels=3, out_channels=in_ch, kernel_size=1), nn.LeakyReLU(0.2) ) )
            self.downscale_blocks.append(DblockPGGAN(in_channels=in_ch, out_channels=out_ch, last_block=False))

    def forward(self,x_rgb):
        x = self.from_rgb[self.depth-1](x_rgb)
        #print(f"from rgb {x.shape}")
        x = self.downscale_blocks[self.depth-1](x)
        #print(f"down 1 {x.shape}")

        if self.alpha < 1.0 and self.depth > 1:
            x_rgb = self.downscale(x_rgb)
            x_old = self.from_rgb[self.depth-2](x_rgb)
            #print(f"x {x.shape}   x old {x_old.shape}")
            x = self.alpha * x + (1-self.alpha) * x_old 
            self.alpha += self.step_alpha
        i = 2
        for block in reversed(self.downscale_blocks[:self.depth-1]):
            x = block(x)
            #print(f"block {i}   {x.shape}")
            i += 1
        return x  

In [ ]:
d = DiscriminatorPGGAN()

In [ ]:
latent_dim = 512

In [ ]:
d.depth = 1
for i in range(2,9):
    img = torch.randn((1,3,2**i,2**i))
    out_d = d(img)
    print(f"out tensor {out_d.shape}")
    d.depth += 1

In [ ]:
d.depth = 1
x_hat = torch.randn((32, 3, 4, 4))
x_hat.requires_grad_(True)
y_hat = d(x_hat)

In [ ]:
grad = torch.autograd.grad(
    outputs=y_hat,
    inputs=x_hat,
    grad_outputs=torch.ones_like(y_hat),
    create_graph=True,
    retain_graph=True,
    only_inputs = True
)[0]

In [ ]:
grad.shape

In [ ]:
y_hat.shape